# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 17 2022 9:27AM,239827,19,8156790,"Bi-Coastal Pharma International, LLC",Released
1,Jun 17 2022 9:25AM,239826,10,SOIVA0000491,Ivaoes Animal Health,Released
2,Jun 17 2022 9:19AM,239825,10,AERO616-01,Aerogen LTD.,Released
3,Jun 17 2022 9:19AM,239825,10,6597676,Aerogen LTD.,Released
4,Jun 17 2022 9:14AM,239824,20,ALU0007087,Alumier Labs Inc.,Released
5,Jun 17 2022 9:14AM,239824,20,ALUR127224794,Alumier Labs Inc.,Released
6,Jun 17 2022 9:14AM,239824,20,ALUR681621027,Alumier Labs Inc.,Released
7,Jun 17 2022 9:14AM,239824,20,ALUR323662594,Alumier Labs Inc.,Released
8,Jun 17 2022 9:14AM,239824,20,ALUR420267796,Alumier Labs Inc.,Released
9,Jun 17 2022 9:14AM,239824,20,ALU0007088,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,239821,Released,4
33,239824,Released,19
34,239825,Released,2
35,239826,Released,1
36,239827,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239821,NaN,7.0,4.0
239824,NaN,NaN,19.0
239825,NaN,NaN,2.0
239826,NaN,NaN,1.0
239827,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239727,0.0,0.0,1.0
239754,0.0,0.0,1.0
239755,0.0,0.0,1.0
239758,1.0,0.0,7.0
239762,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239727,0,0,1
239754,0,0,1
239755,0,0,1
239758,1,0,7
239762,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239727,0,0,1
1,239754,0,0,1
2,239755,0,0,1
3,239758,1,0,7
4,239762,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239727,,,1
1,239754,,,1
2,239755,,,1
3,239758,1,,7
4,239762,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 17 2022 9:27AM,239827,19,"Bi-Coastal Pharma International, LLC"
1,Jun 17 2022 9:25AM,239826,10,Ivaoes Animal Health
2,Jun 17 2022 9:19AM,239825,10,Aerogen LTD.
4,Jun 17 2022 9:14AM,239824,20,Alumier Labs Inc.
23,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation
83,Jun 17 2022 8:59AM,239821,12,Hush Hush
94,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc."
115,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation
116,Jun 17 2022 8:57AM,239819,12,Hush Hush
118,Jun 17 2022 8:54AM,239816,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 17 2022 9:27AM,239827,19,"Bi-Coastal Pharma International, LLC",,,1
1,Jun 17 2022 9:25AM,239826,10,Ivaoes Animal Health,,,1
2,Jun 17 2022 9:19AM,239825,10,Aerogen LTD.,,,2
3,Jun 17 2022 9:14AM,239824,20,Alumier Labs Inc.,,,19
4,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,,,60
5,Jun 17 2022 8:59AM,239821,12,Hush Hush,,7,4
6,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",,,21
7,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,,,1
8,Jun 17 2022 8:57AM,239819,12,Hush Hush,,,2
9,Jun 17 2022 8:54AM,239816,15,"Virtus Pharmaceuticals, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:27AM,239827,19,"Bi-Coastal Pharma International, LLC",1,,
1,Jun 17 2022 9:25AM,239826,10,Ivaoes Animal Health,1,,
2,Jun 17 2022 9:19AM,239825,10,Aerogen LTD.,2,,
3,Jun 17 2022 9:14AM,239824,20,Alumier Labs Inc.,19,,
4,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60,,
5,Jun 17 2022 8:59AM,239821,12,Hush Hush,4,7,
6,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",21,,
7,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,1,,
8,Jun 17 2022 8:57AM,239819,12,Hush Hush,2,,
9,Jun 17 2022 8:54AM,239816,15,"Virtus Pharmaceuticals, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:27AM,239827,19,"Bi-Coastal Pharma International, LLC",1,,
1,Jun 17 2022 9:25AM,239826,10,Ivaoes Animal Health,1,,
2,Jun 17 2022 9:19AM,239825,10,Aerogen LTD.,2,,
3,Jun 17 2022 9:14AM,239824,20,Alumier Labs Inc.,19,,
4,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:27AM,239827,19,"Bi-Coastal Pharma International, LLC",1.0,NaN,NaN
1,Jun 17 2022 9:25AM,239826,10,Ivaoes Animal Health,1.0,NaN,NaN
2,Jun 17 2022 9:19AM,239825,10,Aerogen LTD.,2.0,NaN,NaN
3,Jun 17 2022 9:14AM,239824,20,Alumier Labs Inc.,19.0,NaN,NaN
4,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:27AM,239827,19,"Bi-Coastal Pharma International, LLC",1.0,0.0,0.0
1,Jun 17 2022 9:25AM,239826,10,Ivaoes Animal Health,1.0,0.0,0.0
2,Jun 17 2022 9:19AM,239825,10,Aerogen LTD.,2.0,0.0,0.0
3,Jun 17 2022 9:14AM,239824,20,Alumier Labs Inc.,19.0,0.0,0.0
4,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2877627,95.0,1.0,0.0
12,479640,6.0,7.0,0.0
15,1438783,70.0,0.0,1.0
16,1678427,5.0,8.0,0.0
19,479636,34.0,11.0,0.0
20,479599,29.0,5.0,0.0
21,239790,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2877627,95.0,1.0,0.0
1,12,479640,6.0,7.0,0.0
2,15,1438783,70.0,0.0,1.0
3,16,1678427,5.0,8.0,0.0
4,19,479636,34.0,11.0,0.0
5,20,479599,29.0,5.0,0.0
6,21,239790,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,95.0,1.0,0.0
1,12,6.0,7.0,0.0
2,15,70.0,0.0,1.0
3,16,5.0,8.0,0.0
4,19,34.0,11.0,0.0
5,20,29.0,5.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,95.0
1,12,Released,6.0
2,15,Released,70.0
3,16,Released,5.0
4,19,Released,34.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Executing,1.0,7.0,0.0,8.0,11.0,5.0,1.0
Released,95.0,6.0,70.0,5.0,34.0,29.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,1.0,7.0,0.0,8.0,11.0,5.0,1.0
2,Released,95.0,6.0,70.0,5.0,34.0,29.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,1.0,7.0,0.0,8.0,11.0,5.0,1.0
2,Released,95.0,6.0,70.0,5.0,34.0,29.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()